In [1]:
from policyengine.entities import create_db_and_tables
import h5py
import numpy as np
from sqlmodel import SQLModel, Field, create_engine, Session
from typing import Optional
import pandas as pd
from policyengine.entities import Entity, Variable
import time

# engine = create_db_and_tables(connection_string="sqlite:///tax_policy_sqlite.db")

/opt/miniconda3/envs/py-3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from policyengine_us import Microsimulation

def load_vars_to_json(vars: dict):
    with open('variables.json', 'w') as f:
        for var_name, var in vars.items():
            record = {
                'country_id': 'us',
                'name': var_name,
                'description': getattr(var, 'description', ''),
                'dataset_series_id': 1
            }
            f.write(f"{record}\n")

# Load -us sim just to get variables in structured way
sim: Microsimulation = Microsimulation()

tbs = sim.tax_benefit_system
vars = tbs.variables

start_time = time.time()
load_vars_to_json(vars)
end_time=time.time()
print(f"Variable metadata loaded in {end_time - start_time:.2f} seconds")

Variable metadata loaded in 0.01 seconds


In [3]:

def load_hdf5_to_json(hdf5_path: str, dataset_name: str):
    with h5py.File(hdf5_path, 'r') as hdf5_file:
        dataset = hdf5_file[dataset_name]["2024"]

        with open(f"{dataset_name}_data.json", 'w') as f:
            for i, row in enumerate(dataset):
                record = {
                    'id': i,
                    'country_id': 'us',
                    'entity_type': 'household',
                    'dataset_series_id': 1,  # Assuming a dataset ID of 1 for this example
                }
                f.write(f"{record}\n")

start_time = time.time()
load_hdf5_to_json("./enhanced_cps_2024.h5", "age")
end_time = time.time()
print(f"Households loaded in {end_time - start_time:.2f} seconds")


Households loaded in 0.52 seconds


In [4]:
from policyengine import Simulation

pysim = Simulation(
    country="us",
    scope="macro",
    time_period=2025,
)

pysim.calculate_single_economy()

No data provided, using default dataset: gs://policyengine-us-data/cps_2023.h5
INFO:root:Using Google Cloud Storage for download.
INFO:root:Metadata for blob policyengine-us-data, cps_2023.h5 has version: 1.31.0
INFO:policyengine.utils.data.caching_google_storage_client:Syncing policyengine-us-data, cps_2023.h5, 1.31.0 to cache
INFO:root:Blob policyengine-us-data, /b/policyengine-us-data/o/cps_2023.h5 has version 1.31.0
INFO:policyengine.utils.data.simplified_google_storage_client:Crc for policyengine-us-data, cps_2023.h5 is mW2ZeQ==
INFO:root:Blob policyengine-us-data, /b/policyengine-us-data/o/cps_2023.h5 has version 1.31.0
INFO:policyengine.utils.data.simplified_google_storage_client:Downloaded policyengine-us-data, cps_2023.h5, version:1.31.0
INFO:policyengine.utils.data.caching_google_storage_client:Downloaded new version of policyengine-us-data, cps_2023.h5, 1.31.0 with crc mW2ZeQ==
INFO:policyengine.utils.data.caching_google_storage_client:Cache updated for {id_string}
INFO:poli

Variable 'household_count_people' written to JSON file in 49.27 seconds
Variable 'household_count_people' written to database with strategy 'json'
Variable 'household_tax' written to JSON file in 47.60 seconds
Variable 'household_tax' written to database with strategy 'json'
Variable 'household_benefits' written to JSON file in 45.70 seconds
Variable 'household_benefits' written to database with strategy 'json'
Variable 'equiv_household_net_income' written to JSON file in 45.50 seconds
Variable 'equiv_household_net_income' written to database with strategy 'json'
Variable 'is_male' written to JSON file in 113.06 seconds
Variable 'is_male' written to database with strategy 'json'
Variable 'race' written to JSON file in 120.11 seconds
Variable 'race' written to database with strategy 'json'
Variable 'age' written to JSON file in 124.83 seconds
Variable 'age' written to database with strategy 'json'
Variable 'in_poverty' written to JSON file in 54.35 seconds
Variable 'in_poverty' written 

AttributeError: 'NoneType' object has no attribute 'id'